In [ ]:
import pandas as pd
data_df = pd.read_csv('data/hourly_wages.csv')

In [ ]:
data_df.head()

In [ ]:
data_df.describe()

In [ ]:
target = data_df.wage_per_hour.as_matrix()
predictors = data_df.drop(['wage_per_hour'], axis=1).as_matrix()

In [ ]:
n_cols = predictors.shape[1]

## My First Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

my_model = Sequential()
my_model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
my_model.add(Dense(100, activation='relu'))
my_model.add(Dense(1))

my_model.summary()

In [ ]:
my_model.compile(optimizer = 'adam', loss='mean_squared_error')

In [ ]:
my_model.fit(predictors, target)

## Validation and Early Stopping

In [ ]:
def get_new_model():
    my_model = Sequential()
    my_model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    my_model.add(Dense(100, activation='relu'))
    my_model.add(Dense(1))
    my_model.compile(optimizer = 'adam', loss='mean_squared_error')
    return(my_model)
my_model = get_new_model()
my_model.fit(predictors, target, validation_split=0.3)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=2)
my_model = get_new_model()
my_model.fit(predictors, target, validation_split=0.3, nb_epoch=20, callbacks=[early_stopping_monitor])

## Classification

In [ ]:
titanic_data = pd.read_csv('data/titanic_all_numeric.csv')
titanic_data.head()

In [ ]:
from keras.utils.np_utils import to_categorical
target = to_categorical(titanic_data.survived)
predictors = titanic_data.drop(['survived'], axis=1).as_matrix()
n_cols = predictors.shape[1]

def get_classification_model(n_cols):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

my_model = get_classification_model(n_cols)
my_model.fit(predictors, target, validation_split=0.3, nb_epoch=20, callbacks=[early_stopping_monitor])

## Convolutional Networks (Working with Images)

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D

nb_classes = 10

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# input image dimensions
img_rows, img_cols = 28, 28

# Using tf dim ordering
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)


In [ ]:
# number of convolutional filters to use
nb_filters = 16
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, Y_train, nb_epoch=5, validation_data=(X_test, Y_test))

## Transfer Learning (using Pre-Trained Networks)
From https://github.com/fchollet/deep-learning-models

In [1]:
from vgg16 import VGG16
from keras.preprocessing import image
from imagenet_utils import preprocess_input

model = VGG16(weights='imagenet', include_top=False)


#img = image.load_img(img_path, target_size=(224, 224))
#x = image.img_to_array(img)
#x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

#features = model.predict(x)

Using TensorFlow backend.


K.image_dim_ordering: tf


ResourceExhaustedError: OOM when allocating tensor with shape[512]
	 [[Node: block4_conv1_b/Assign = Assign[T=DT_FLOAT, _class=["loc:@block4_conv1_b"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](block4_conv1_b, Const_7)]]

Caused by op 'block4_conv1_b/Assign', defined at:
  File "/Users/dan/anaconda/envs/py35/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/dan/anaconda/envs/py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/dan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/dan/anaconda/envs/py35/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/dan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/